In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import drone_sar

In [42]:
from drone_sar.ipsar_dataset import IPSARDataset
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [5]:
DS_ROOT = "/Users/iz/Downloads/heridal/"

In [6]:
# !ls {DS_ROOT}

In [23]:
train_data = IPSARDataset("/Users/iz/Downloads/heridal/trainImages")
test_data = IPSARDataset("/Users/iz/Downloads/heridal/testImages")

In [24]:
item = test_data[0]

In [25]:
item.keys()

dict_keys(['img_path', 'pil', 'pixel_values', 'boxes', 'class_labels'])

In [26]:
item['pixel_values'].shape

torch.Size([3000, 4000, 3])

In [27]:
item["boxes"].shape

torch.Size([2, 4])

In [28]:
item["class_labels"].shape

torch.Size([2])

In [29]:
# data[0]

In [30]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

In [21]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /Users/iz/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


In [36]:
# item['pil']

In [41]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = item['pil']

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [33]:
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)[0]

In [ ]:
results = []
for item in tqdm(train_data + test_data):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    target_sizes = torch.tensor([image.size[::-1]])
    res = processor.post_process_object_detection(
        outputs, target_sizes=target_sizes, threshold=0.5
    )[0]
    results.append(res)

  0%|          | 0/1684 [00:00<?, ?it/s]